# Riddler Classic - [December 18, 2020](https://fivethirtyeight.com/features/en-garde-can-you-win-the-fencing-relay/)

## Problem
_TYou are the coach at Riddler Fencing Academy, where your three students are squaring off against a neighboring squad. Each of your students has a different probability of winning any given point in a match. The strongest fencer has a 75 percent chance of winning each point. The weakest has only a 25 percent chance of winning each point. The remaining fencer has a 50 percent probability of winning each point._

_The match will be a relay. First, one of your students will face off against an opponent. As soon as one of them reaches a score of 15, they are both swapped out. Then, a different student of yours faces a different opponent, continuing from wherever the score left off. When one team reaches 30 (not necessarily from the same team that first reached 15), both fencers are swapped out. The remaining two fencers continue the relay until one team reaches 45 points._

_As the coach, you can choose the order in which your three students occupy the three positions in the relay: going first, second or third. How will you order them? And then what will be your team’s chances of winning the relay?_

## Solution

In first attempting the problem, I initally thought there may be some way to solve it analytically if I could find a convenient way to calculate the probability of winning without needing to account for the complexities of inter-round dependence.  More specifically, I explored whether there was a way to calculate 
$$ P(R_3) = P(R_3 | R_2) + P(R_3 | ~ R_2) $$ 
where $R_i$ is the event that your team wins round $i$.

After playing around with this idea for a while, I realized that I wasn't finding any tricks that would allow me to isolate the rounds from one another, which I thought  would greatly simplify the calculation. Instead, I was left with terms like $ P(R_{i+1} and R_i) $ which, after trying to calculate for i=1, I quickly realized was beyond my combinatorics abilities. 

So, I pivoted to a computational approach.  Looking at the figure below, we can start to get some intuition for how we can solve this problem. 

![Scoring Graph](./classic_2021-12-10_graph.png)

We know that if you or your opponent scores 45 points, the match is over.  Let $ P(i,j) $ be the probability that you win when you have $i$ points and your opponent has $j$ points. So $ P(i=45,j)=1 $ and $ P(i,j=45)=0 $. If we can find a way to go backwards from this to get to $ P(i=0, j=0) $, which is our answer. Using the intuition from the graph, we have the probabilities along the red and the green lines.  We just need to calculate the probabilities backwards from there.  

We can observe that there are two outcomes for a game with a score of $ i, j $. First, we can wind a round wiht probability $p$ resulting in $ i+1, j $. Second our opponent can win a round with probability $1-p$ resulting in $ i, j-1 $. Since we know the probability that we (or our opponent) win any given round, we can write the recursive relationship

$$ P(i,j) = p * P(i+1,j) + (1-p) * P(i,j+1) $$

where $ p $ is the probability we win a given round. 

Equipped with this recurrence relation and our boundary conditions at $i=45$ and $j=45$, we have everything we need to iteritively calculate backward to get $ P(i=0, j=0) $. The following code calculates that value.

In [9]:
def calculation(percentages, round_cutoffs):
    """calculates the probability of your team winning the game described above
    
    Parameters:
        percentages (list): likehood your player wins a given game 
        round_cutoffs (list of ints): number of games each set of competitors must win before starting the next round

    Returns:
        p
    """

    def which_round(n, m):
        i = 0
        while True:
            if n < round_cutoffs[i] and m < round_cutoffs[i]:
                return i

    import numpy as np

    total_rounds = round_cutoffs[-1]

    probabilities = np.zeros([total_rounds, total_rounds-1])
    
    # Set boundary conditions
    probabilities[-1, :] = 1

    for i in range(len(probabilities[:,0])-1, -1):
        for j in range(len(probabilities[0.,:]), -1):
            round_num = which_round(i,j) 
            p = percentages[round_num]
            probabilities[i, j] = (probabilities[i+1, j] - (1-p) * probabilities[i+1, j-1]) / p
    
    return probabilities[0][0]


In [11]:
perc = [.25, .5, .75]
rounds = [15, 30, 45]
calculation(perc, rounds)

0.0